<a href="https://colab.research.google.com/github/OksanaGlazkova/Numpy-pandas/blob/main/%D0%94_%D0%97_%D0%9E%D1%81%D0%BD%D0%BE%D0%B2%D1%8B_%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%D0%B0_%D0%B8_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%8B_%D1%81_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание 1.** Обязательная часть Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = [‘python’, ‘парсинг’]

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная) Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import time

In [ ]:
import pandas as pd

In [ ]:
KEYWORDS = ['python', 'парсинг']

In [ ]:
params = 'https://habr.com/ru/all/'

In [ ]:
res = requests.get(params)

In [ ]:
import re

In [ ]:
def get_all_links(url):
    params = 'https://habr.com/ru/all/'
    
    res = requests.get(params)
    soup = BeautifulSoup(res.text)
    posts = soup.find_all('div', class_='tm-article-body')
    a_list = list(map(lambda x: x.find('a').get('href'), posts))
    all_refs = list(map(lambda x: 'https://habr.com' + x, a_list))
    all_refs_post = re.findall(r'(https://habr.com/ru/post/\d{6}/)', str(all_refs))

    return all_refs_post

all_links = get_all_links('https://habr.com/ru/all/')
all_links

['https://habr.com/ru/post/587076/',
 'https://habr.com/ru/post/586788/',
 'https://habr.com/ru/post/585828/',
 'https://habr.com/ru/post/587054/',
 'https://habr.com/ru/post/587044/',
 'https://habr.com/ru/post/469247/',
 'https://habr.com/ru/post/587014/',
 'https://habr.com/ru/post/587006/',
 'https://habr.com/ru/post/586986/']

In [ ]:
def get_word(url):
  word = pd.DataFrame()
  for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    if soup.find('span', class_='tm-article-snippet__datetime-published'):
      date = pd.to_datetime(soup.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True).date()
      time.sleep(0.3)
    if soup.find('h2', class_='tm-article-snippet__title'):
        title = soup.find('h2', class_='tm-article-snippet__title').text
        time.sleep(0.3)
    else:
        title = soup.find('h1', class_='tm-article-snippet__title').text
    text = soup.find('div', class_='tm-article-body').text
    for w in text:
      w_lower = w.lower()
      if any([w_lower in desired for desired in KEYWORDS]):
        text == text
    row = {'date': date, 'title': title, 'text': text}
    word = pd.concat([word, pd.DataFrame([row])])  
  return word

habr = get_word(all_links)
habr

,date,title,text
0,2021-11-02,Московский Сайлент-Хилл: естественного происхо...,С вечера понедельника и по текущий момент не у...
0,2021-11-02,Unreal engine — UMG Best Practices,"Это перевод статьи Chris Gagnon, сотрудника Ep..."
0,2021-11-02,NFT: особенности и возможности. OpenSea — плат...,\n\r\nНа сегодняшний день в связи с пандемией ...
0,2021-11-02,Инвалидное кресло-каталка на электротяге,Два года назад помогал другу с постройкой элек...
0,2021-11-02,"5 причин, почему не стоит становиться программ...","Достаточно людей очарованы программистами, и я..."
0,2021-03-04,Простой способ получения ”Flicker-Off”: «отклю...,"''Свобода* лучше, чем несвобода*''©\n*от мерца..."
0,2021-11-02,"Работа с данными. FOR-архитектура, часть 1","Сразу извиняюсь за свою манеру повествования, ..."
0,2021-11-02,MARVEL: интеграция со сторонней учетной системой,Немного о Marvel и его прошломКомпания «Марвел...
0,2021-11-02,HDR-видео и смартфон,"Каждый, кто хоть чуть-чуть интересуется технол..."


А можно ли так ещё? Ещё один вариант поиска по df:

In [ ]:
habr[habr.text.str.contains(str(KEYWORDS))].head()

,date,title,text
0,2021-11-02,Московский Сайлент-Хилл: естественного происхо...,С вечера понедельника и по текущий момент не у...
0,2021-11-02,Unreal engine — UMG Best Practices,"Это перевод статьи Chris Gagnon, сотрудника Ep..."
0,2021-11-02,NFT: особенности и возможности. OpenSea — плат...,\n\r\nНа сегодняшний день в связи с пандемией ...
0,2021-11-02,Инвалидное кресло-каталка на электротяге,Два года назад помогал другу с постройкой элек...
0,2021-11-02,"5 причин, почему не стоит становиться программ...","Достаточно людей очарованы программистами, и я..."


**Задание 2.**
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.

In [ ]:
import requests

In [ ]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [ ]:
params = {
    'emailAddresses': EMAIL
    }

In [ ]:
print(params)

{'emailAddresses': ['xxx@x.ru', 'yyy@y.com']}


In [ ]:
headers = {
    'Vaar-Header-App-Build-Version' : '1.0.0',
    'Vaar-Header-App-Product' : 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name' : 'hackcheck-web-avast',
    'Vaar-Version' : '0'
}

In [ ]:
req = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data', headers=headers, json=params)
req

<Response [200]>

In [ ]:
req.text

'{"breaches":{"2":{"breachId":2,"site":"linkedin.com","recordsCount":158591429,"description":"In 2012, online professional networking platform LinkedIn fell victim to a breach of its members\' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \\n\\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":111975337,"emails":158591429}},"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion w

In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
req.json()['breaches']

{'12': {'breachId': 12,
  'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a dark web marketplace as of June 2016 for a price of one bitcoin. ",
  'publishDate': '2016-10-29T00:00:00Z',
  'recordsCount': 110121799,
  'site': 'vk.com',
  'statistics': {'emails': 104247327, 'passwords': 107425364, 'usernames': 0}},
 '13094': {'breachId': 13094,
  'description': 'In October 2017, a customer database belonging to online genealogy platform MyHeritage was breached. The stolen records included email addresses and SHA-1 hashed passwords, as well as sensitive PII. The data has been sold on at least one dark web market and is also being privately shared amongst several criminal groups.',
  'publishDate': '2017-11-04T00:00:

In [ ]:
r = req.json()['breaches']
for key, value in r.items():
  val = pd.DataFrame(value)
  
print(val[['publishDate', 'site', 'description']])


                    publishDate  ...                                        description
emails     2021-02-11T00:00:00Z  ...  At an unconfirmed date, the Russian-language m...
passwords  2021-02-11T00:00:00Z  ...  At an unconfirmed date, the Russian-language m...
usernames  2021-02-11T00:00:00Z  ...  At an unconfirmed date, the Russian-language m...

[3 rows x 3 columns]


Кривовато, конечно, получилось)))

Спасибо за внимание!☺